# Setup

In [ ]:
from pathlib import Path
import cv2
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

# TensorFlow Autoencoder

## Getting the data

In [ ]:
def get_data(directory='../datasets/raw/renders_2d/images/', color=False):
    
    # count nr. of files to initialize numpy array
    _, _, files = next(os.walk(directory))
    file_count = len(files)

    if color:
        imgs = np.zeros(shape=(file_count, 256, 256, 3))
    else:
        imgs = np.zeros(shape=(file_count, 256, 256))
    
    # iterate over files in that directory
    files = Path(directory).glob('*')
    for idx, file in enumerate(files):
        if color:
            imgs[idx] = cv2.imread(str(file)) # color
        else:
            imgs[idx] = cv2.imread(str(file), 0) # grayscale
    
    return imgs

In [ ]:
imgs = get_data(color=True)

In [ ]:
print(imgs.shape)
print(imgs[0, :, :].shape)
print(imgs[0, 128, 128])

In [ ]:
x_train, x_test = train_test_split(imgs, test_size=0.2)

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

print(x_train.shape)
print(x_test.shape)

print(x_train.shape)
print(x_train[0, :, :].shape)
print(x_train[0, 128, 128, :])

print(np.count_nonzero(np.isnan(x_train)))

## Defining the model

In [ ]:
class Autoencoder(Model):
  def __init__(self, latent_dim, shape):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim
    self.shape = shape
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(latent_dim, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(tf.math.reduce_prod(shape), activation='sigmoid'),
      layers.Reshape(shape)
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded


shape = x_test.shape[1:]
print(shape)
latent_dim = 640 # hyperparam
autoencoder = Autoencoder(latent_dim, shape)

In [ ]:
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

## Training the model
Train the model using x_train as both the input and the target. The encoder will learn to compress the dataset from 784 dimensions to the latent space, and the decoder will learn to reconstruct the original images.

In [ ]:
autoencoder.fit(x_train, x_train,
                epochs=100, # hyperparam
                shuffle=True,
                validation_data=(x_test, x_test))

In [ ]:
encoded_imgs = autoencoder.encoder(x_test).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

In [ ]:
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
  # display original
  ax = plt.subplot(2, n, i + 1)
  plt.imshow(x_test[i])
  plt.title("original")
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

  # display reconstruction
  ax = plt.subplot(2, n, i + 1 + n)
  plt.imshow(decoded_imgs[i])
  plt.title("reconstructed")
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()